In [419]:
import collections
import itertools
import json
import re
import struct

import networkx as nx
import requests

In [512]:
ELEMENT_MATCHER = r'((Dr|Mr|Mrs|Prof).|\b[a-zA-Z\']+\b|;|:|,|\.|\?|!)'  # whole words and some punctuation

In [513]:
QUOTE_TRANS = str.maketrans({
    '\N{LEFT SINGLE QUOTATION MARK}': "'",
    '\N{RIGHT SINGLE QUOTATION MARK}': "'",
    '\N{LEFT DOUBLE QUOTATION MARK}': '"',
    '\N{RIGHT DOUBLE QUOTATION MARK}': '"',
})

In [497]:
def words(*, filename=None, text=None, harmonize_caps=True, trans=None):
    if filename:
        with open(filename, encoding='utf-8') as f:
            lines = [line for line in f if not line.isupper()]
    else:
        lines = text.splitlines()

    if trans is None:
        trans = QUOTE_TRANS
        
    for line in lines:
        line = line.translate(trans)
        yield from line.split()

In [ ]:
def genwords2(*args, **kwargs):
    # initial cleaning/fragmenting
    pass1 = words(*args, **kwargs)
    
    # find most common capitalizations
    most_common_capitalization()

In [514]:
def gen_words(*, filename=None, text=None, harmonize_caps=True):
    if filename:
        with open(filename, encoding='utf-8') as f:
            lines = [line for line in f if not line.isupper()]
    else:
        lines = text.splitlines()
        
    if harmonize_caps:
        translator = most_common_capitalization(gen_words(filename=filename, text=text, harmonize_caps=False))
    else:
        translator = {}
        
    for line in lines:
        line = line.translate(QUOTE_TRANS)
        for word in re.findall(ELEMENT_MATCHER, line):
            word = word[0]
            yield translator.get(word, word)
            

def most_common_capitalization(words):
    case_sensitive = collections.Counter(words)
    case_insensitive = collections.defaultdict(dict)
    for word, count in case_sensitive.items():
        case_insensitive[word.lower()][word] = count
        
    translator = {}    
    for lower, counts in case_insensitive.items():
        if len(counts) < 2:
            continue
        winner = max(counts.items(), key=lambda x: x[1])[0]
            
        for variant in counts:
            translator[variant] = winner

    return translator

In [510]:
start_marker = '*** START OF THIS PROJECT GUTENBERG EBOOK DRACULA ***'
end_marker = '*** END OF THIS PROJECT GUTENBERG EBOOK DRACULA ***'

dracula = requests.get('http://www.gutenberg.org/cache/epub/345/pg345.txt').text
dracula = dracula[dracula.find(start_marker) + len(start_marker):dracula.find(end_marker)]

In [515]:
drac_corpus = list(gen_words(text=dracula))[231:]
drac_counter = collections.Counter(drac_corpus)

In [516]:
per_line = 7
for n, (block, count) in enumerate(drac_counter.most_common(200)):
    print('{:>4d} {:<8s}'.format(count, block), end='')
    if n % per_line == per_line - 1:
        print()

11243 ,       7999 .       7907 the     5905 and     4800 I       4663 to      3629 of      
2954 a       2567 he      2507 in      2469 that    2156 it      1879 was     1684 ;       
1590 as      1543 we      1537 for     1501 is      1471 his     1455 me      1402 not     
1397 you     1286 with    1261 my      1164 all     1115 be      1107 so      1083 at      
1073 on      1071 but     1060 have    1059 her     1039 had      953 him      813 she     
 776 when     773 there    751 !        663 which    638 if       636 :        631 this    
 623 from     582 are      570 said     552 were     549 then     518 by       496 one     
 492 ?        491 could    488 no       474 do       472 them     463 or       463 they    
 463 what     462 us       454 will     448 up       442 some     440 must     432 would   
 430 out      426 shall    414 may      407 our      407 now      395 see      391 know    
 391 been     390 time     377 can      376 more     359 an       345 has      

In [517]:
iwords = iter(drac_corpus)
prev = next(iwords)
G = nx.DiGraph()
for word in iwords:
    if G.has_edge(prev, word):
        G[prev][word]['weight'] += 1
    else:
        G.add_edge(prev, word, weight=1)
    prev = word

In [95]:
nld = nx.readwrite.json_graph.node_link_data(G)
# nld.pop('directed'), nld.pop('multigraph')
len(json.dumps(nld))

3648988

In [102]:
adjd = nx.readwrite.json_graph.adjacency_data(G)
print(len(json.dumps(adjd)))

print(adjd.pop('directed'), adjd.pop('multigraph'))
{k: len(adjd[k]) for k in adjd}

2157962
True False


{'adjacency': 9334, 'graph': 0, 'nodes': 9334}

In [109]:
import math

In [230]:
import struct

In [242]:
struct.unpack_from('4s', b'abcd\0')

(b'abcd',)

In [273]:
hex(unpack_varint([129, 128, 128, 0]))

'0x200000'

In [352]:
def pack_varuint(n):
    if n < 0:
        raise ValueError('value must be non-negative')
    packed = [n & 0x7F]
    n >>= 7
    while n > 0:
        packed.insert(0, 0x80 | (n & 0x7F))
        n >>= 7

    return packed

def unpack_varuint(byteiter):
    n = 0
    for byte in byteiter:
        n <<= 7
        n += byte & 0x7F
        if not (byte & 0x80):
            return n
    raise ValueError('byte iterator exhausted')
        
def test_unpack_varuint():
    assert unpack_varuint(b'\x00') == 0
    assert unpack_varuint(b'\x00\x00') == 0
    assert (unpack_varuint(b'\x81\x00') ==
            unpack_varuint(b'\x81\x00\x00') ==
            128)
    
    bi = iter(b'\x00\x00')
    assert unpack_varuint(bi) == 0
    assert unpack_varuint(bi) == 0

def test_packunpack_varuint():
    big = 1234567890212345678902345432
    nums = itertools.chain(
        range(10000), 
        range(0x3FF0, 0x400F), 
        range(0x1FFFF0, 0x20000F),
        range(big, big + 1234),
    )
    for n in nums:
        packed = pack_varuint(n)
        assert n == unpack_varuint(packed)
        assert not (packed[-1] & 0x80)
        assert all(px & 0x80 for px in packed[:-1])
        
test_packunpack_varuint()
test_unpack_varuint()

In [252]:
b'321' + bytes([53])

b'3215'

In [253]:
import itertools

In [296]:
def pack_str(s):
    b = s.encode('utf-8')
    return bytes([len(b)]) + bytes(b)

def unpack_str(byteiter):
    byteiter = iter(byteiter)
    n = next(byteiter)
    b = bytes(itertools.islice(byteiter, n))
    return b.decode('utf-8')

def test_packunpack_str():
    for word in itertools.__doc__.split():
        unpack_str(pack_str(word)) == word
        
test_packunpack_str()

In [398]:
def pump(type_):
    '''
    Feed the generator into *type_* til exhaustion. Can be overridden by
    setting exhaust=False.
    '''
    def _pump(f):
        def fx(*args, **kwargs):
            exhaust_gen = kwargs.pop('exhaust', True)
            gen = f(*args, **kwargs)
            if exhaust_gen:
                return type_(gen)
            return gen
        return fx
    return _pump

@pump(bytes)
def pack_strarr(strs):
    yield from pack_varuint(len(strs))
    for s in strs:
        yield from pack_str(s)
    
@pump(list)
def unpack_strarr(byteiter):
    byteiter = iter(byteiter)
    n = unpack_varuint(byteiter)
    for _ in range(n):
        yield unpack_str(byteiter)
    
def test_pack_strarr():
    assert pack_strarr([]) == b'\x00'
    assert pack_strarr(['hey', 'there', '\N{UNICORN FACE}']) == b'\x03\x03hey\x05there\x04\xf0\x9f\xa6\x84'
    
def test_unpack_strarr():
    assert unpack_strarr(b'\x00') == []
    assert unpack_strarr(b'\x01\x01x') == ['x']
    assert unpack_strarr(b'\x01\x01x' + bytes(100)) == ['x']
    
def test_packunpack_strarr():
    def pup(x):
        return unpack_strarr(pack_strarr(x))
    a1 = ['x'] * 10000
    assert pup(a1) == a1
    
    x = ord('\N{SLIGHTLY SMILING FACE}')
    emojis = [chr(n) for n in range(x, x + 130)]
    assert pup(emojis) == emojis
    
test_pack_strarr()
test_unpack_strarr()
test_packunpack_strarr()

In [399]:
@pump(bytes)
def pack_intarr_2d(arr):
    rows = len(arr)
    if rows == 0:
        yield from b'\x00\x00'
        return
    yield from pack_varuint(rows)
    cols = len(arr[0])
    yield from pack_varuint(cols)
    for row in arr:
        if len(row) != cols:
            raise ValueError('jagged array')
        for element in row:
            yield from pack_varuint(element)

def test_pack_intarr():
    assert pack_intarr_2d([]) == b'\x00\x00'
    assert pack_intarr_2d([[]]) == b'\x01\x00'
    assert pack_intarr_2d([[], []]) == b'\x02\x00'
    assert pack_intarr_2d([[1]]) == bytes([1] * 3)
    assert pack_intarr_2d([[2, 2], [2, 2]]) == bytes([2] * 6)
    assert pack_intarr_2d([[3] * 3] * 3) == bytes([3] * 11)

def test_pack_intarr_bad():
    try:
        pack_intarr_2d([1, 2])
    except (TypeError, ValueError):
        pass
    else:
        assert False, 'allowed bad array'
        
    try:
        pack_intarr_2d([[1], [2, 2]])
    except (TypeError, ValueError):
        pass
    else:
        assert False, 'allowed bad array'
    
test_pack_intarr()
test_pack_intarr_bad()

@pump(list)
def unpack_intarr_2d(byteiter):
    byteiter = iter(byteiter)
    rows = unpack_varuint(byteiter)
    cols = unpack_varuint(byteiter)
    for row in range(rows):
        yield [unpack_varuint(byteiter) for _ in range(cols)]
        
def test_unpack_intarr():
    assert unpack_intarr_2d(b'\x00\x00') == []
    assert unpack_intarr_2d(b'\x01\x00') == [[]]
    assert unpack_intarr_2d(b'\x02\x00') == [[], []]
    
    assert unpack_intarr_2d(b'\x01\x01\x0F') == [[15]]
    assert unpack_intarr_2d(bytes([2, 2, 1, 2, 3, 4])) == [[1, 2], [3, 4]]
    assert unpack_intarr_2d(bytes([1, 2, 3, 4, 5, 6])) == [[3, 4]]
    
    byteiter = iter(bytes([1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]))
    assert unpack_intarr_2d(byteiter) == [[3, 1]]
    assert unpack_intarr_2d(byteiter) == [[1, 2, 3], [1, 2, 3]]
    assert unpack_intarr_2d(byteiter) == [[3, 1]]
    
test_unpack_intarr()

In [311]:
list(unpack_strarr(b'\x01\x02hi')) == ['hi']

True

In [464]:
def serialize_digraph(G):
    adjd = nx.readwrite.json_graph.adjacency_data(G)
    return serialize_adjd(adjd)

@pump(bytes)
def serialize_adjd(adjd):
    if not adjd['directed'] or adjd['multigraph']:
        raise ValueError('only supports directed simple graphs')
        
    nodes = [n['id'] for n in adjd['nodes']]
    lookup = {nodeword: n for n, nodeword in enumerate(nodes)}
    
    yield from pack_strarr(nodes)

    for adj in adjd['adjacency']:
        node_adj = [[lookup[edge['id']], edge['weight']] for edge in adj]
        yield from pack_intarr_2d(node_adj)
        
def deserialize_digraph(byteiter):
    data = deserialize_adjd(byteiter)
    return nx.readwrite.json_graph.adjacency_graph(data)

def deserialize_adjd(byteiter):
    byteiter = iter(byteiter)
    nodes = unpack_strarr(byteiter)
    
    adjd = []
    for node in nodes:
        adj = unpack_intarr_2d(byteiter)
        adjd.append([{'id': node, 'weight': weight} for id_, weight in adj])
        
    data = {
        'directed': True, 
        'multigraph': False, 
        'nodes': [{'id': n} for n in nodes],
        'adjacency': adjd,
    }
    return data

In [572]:
def assert_fragment(it, fragment):
    assert fragment == bytes(itertools.islice(it, len(fragment)))
        
def test_serialize_adjd():
    nodes = [{'id': 'x'}, {'id': 'y'}]
    adj = [[{'id': 'y', 'weight': 2}], [{'id': 'x', 'weight': 3}]]
    g = {'nodes': nodes, 'adjacency': adj, 'multigraph': False, 'directed': True}
    git = iter(serialize_adjd(g))

    assert_fragment(git, b'\x02\x01x\x01y')  # nodes
    assert_fragment(git, b'\x01\x02\x01\x02')  # node x adjacency
    assert_fragment(git, b'\x01\x02\x00\x03')  # node y adjacency
    assert next(git, None) is None  # should be exhausted
    
def test_serialize_adjd2():
    nodes = [{'id': 'x'}, {'id': 'y'}, {'id': 'zz'}]
    adj = [
        [{'id': 'y', 'weight': 2}, {'id': 'zz', 'weight': 0x44}], 
        [{'id': 'x', 'weight': 3}, {'id': 'zz', 'weight': 0x55}],
    ]
    g = {'nodes': nodes, 'adjacency': adj, 'multigraph': False, 'directed': True}
    git = iter(serialize_adjd(g))
#     gb = serialize_adjd(g)
    assert_fragment(git, b'\x03\x01x\x01y\x02zz')  # nodes
    assert_fragment(git, b'\x02\x02\x01\x02\x02\x44')  # node x adjacency
    assert_fragment(git, b'\x02\x02\x00\x03\x02\x55')  # node y adjacency
    assert next(git, None) is None  # should be exhausted
    
test_serialize_adjd()
test_serialize_adjd2()

In [579]:
def test_deserialize_adjd():
    nodes = [{'id': 'x'}, {'id': 'y'}]
    adj = [[{'id': 'y', 'weight': 2}], [{'id': 'x', 'weight': 3}]]
    g = {'nodes': nodes, 'adjacency': adj, 'multigraph': False, 'directed': True}

    bit = iter(
        b'\x02\x01x\x01y' +  # nodes
        b'\x01\x02\x01\x02' +  # node x adjacency
        b'\x01\x02\x00\x03')  # node y adjacency
    
    G = deserialize_adjd(bit) 
    assert next(bit, None) is None  # should be exhausted
    
    assert G['nodes'] == nodes
    assert G['adjacency'] == adj, G['adjacency']
#     assert set(G.edges()) == 
    
def test_deserialize_adjd2():
    nodes = [{'id': 'x'}, {'id': 'y'}, {'id': 'z'}]
    adj = [
        [{'id': 'y', 'weight': 2}, {'id': 'z', 'weight': 0x44}], 
        [{'id': 'x', 'weight': 3}, {'id': 'z', 'weight': 0x55}],
    ]
    g = {'nodes': nodes, 'adjacency': adj, 'multigraph': False, 'directed': True}

    bit = iter(
        b'\x02\x01x\x01y' +  # nodes
        b'\x01\x02\x01\x02' +  # node x adjacency
        b'\x01\x02\x00\x03')  # node y adjacency
    
    print(deserialize_adjd(bit)) 
    assert next(bit, None) is None  # should be exhausted
    
test_deserialize_adjd()
test_deserialize_adjd2()

AssertionError: [[{'id': 'x', 'weight': 2}], [{'id': 'y', 'weight': 3}]]

In [565]:
import contextlib
import importlib

COMPRESSORS = {lib: None for lib in ['gzip', 'bz2', 'lzma']}
for lib in COMPRESSORS:
    with contextlib.suppress(ImportError):
        COMPRESSORS[lib] = importlib.import_module(lib)
        
COMPRESSED_EXTS = {
    'gz': 'gzip',
    'bz2': 'bz2',
    'xz': 'lzma',
    'lzma': 'lzma',
}
def opener(fn, *args, **kwargs):
    final_ext = fn.rsplit('.')[-1]
    if final_ext in COMPRESSED_EXTS:
        compressor = COMPRESSED_EXTS[final_ext]
        compressor_lib = COMPRESSORS.get(compressor, None)
        if compressor_lib is None:
            raise RuntimeError('could not import library "{}"'.format(compressor))
        return compressor_lib.open(fn, *args, **kwargs)
    return open(fn, *args, **kwargs)

def dump_word_digraph(G, filename):
    dump = serialize_digraph(G)
    with opener(filename, 'wb') as f:
        f.write(dump)
        
def load_word_digraph(filename):
    with opener(filename, 'rb') as f:
        dump = f.read()
    G = deserialize_digraph(dump)
    return G

In [560]:
dump_word_digraph(G, 'myfile.dump')

uncompressed


In [566]:
Gbz = load_word_digraph('myfile.bz2')

In [569]:
set(G.edges()) == set(Gbz.edges())

False

In [571]:
len(G.edges()), len(Gbz.edges())

(66633, 9335)

In [469]:
bit = iter(
    b'\x02\x01x\x01y' +  # nodes
    b'\x01\x02\x01\x02' +  # node x adjacency
    b'\x01\x02\x00\x03')  # node y adjacency
tg = deserialize_digraph(bit)
len(tg)

2

In [484]:
import gzip

In [485]:
sGgz = gzip.compress(sG)

In [476]:
len(dracula)

863315

In [486]:
len(sGgz)

177123

In [458]:
sG = serialize_digraph(G)
len(sG)

270012

In [461]:
len(G)

9334

In [459]:
sG[:100]

b"\xc8v\x08Jonathan\x08Harker's\x07Journal\x02in\tshorthand\x01.\x03may\x08Bistritz\x04left\x06Munich\x02at\x01:\x01P\x01m\x01,\x02on\x08arriving\x06Vienna\x05e"

In [462]:
Gx = deserialize_digraph(iter(sG))

unpacked 9334 nodes


In [283]:
adjd['nodes'][:10]

[{'id': 'Jonathan'},
 {'id': "Harker's"},
 {'id': 'Journal'},
 {'id': 'in'},
 {'id': 'shorthand'},
 {'id': '.'},
 {'id': 'may'},
 {'id': 'Bistritz'},
 {'id': 'left'},
 {'id': 'Munich'}]

In [282]:
adjd['adjacency'][:3]

[[{'id': "Harker's", 'weight': 8},
  {'id': 'Harker', 'weight': 10},
  {'id': 'Nay', 'weight': 1},
  {'id': ',', 'weight': 37},
  {'id': 'some', 'weight': 1},
  {'id': 'from', 'weight': 1},
  {'id': 'and', 'weight': 9},
  {'id': '.', 'weight': 14},
  {'id': 'for', 'weight': 2},
  {'id': 'is', 'weight': 6},
  {'id': ';', 'weight': 3},
  {'id': 'since', 'weight': 1},
  {'id': 'was', 'weight': 6},
  {'id': 'been', 'weight': 1},
  {'id': '!', 'weight': 2},
  {'id': 'awakes', 'weight': 1},
  {'id': 'woke', 'weight': 2},
  {'id': 'wants', 'weight': 1},
  {'id': 'asks', 'weight': 1},
  {'id': 'feels', 'weight': 1},
  {'id': 'tries', 'weight': 1},
  {'id': 'will', 'weight': 3},
  {'id': 'sleeping', 'weight': 1},
  {'id': 'away', 'weight': 1},
  {'id': 'a', 'weight': 2},
  {'id': 'with', 'weight': 2},
  {'id': 'thought', 'weight': 1},
  {'id': 'clutch', 'weight': 1},
  {'id': 'kept', 'weight': 2},
  {'id': 'why', 'weight': 1},
  {'id': 'still', 'weight': 1},
  {'id': 'rising', 'weight': 1},
  {

In [88]:
{k: len(nld[k]) for k in nld}

{'graph': 0, 'links': 66615, 'nodes': 9336}

3648949

In [89]:
nld['links'][:10]

[{'source': 1, 'target': 2, 'weight': 3},
 {'source': 'Jonathan', 'target': "Harker's", 'weight': 8},
 {'source': 'Jonathan', 'target': 'Harker', 'weight': 10},
 {'source': 'Jonathan', 'target': 'Nay', 'weight': 1},
 {'source': 'Jonathan', 'target': ',', 'weight': 37},
 {'source': 'Jonathan', 'target': 'some', 'weight': 1},
 {'source': 'Jonathan', 'target': 'from', 'weight': 1},
 {'source': 'Jonathan', 'target': 'and', 'weight': 9},
 {'source': 'Jonathan', 'target': '.', 'weight': 14},
 {'source': 'Jonathan', 'target': 'for', 'weight': 2}]